In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [2]:
gmsh.open("test-twist-3D.geo")

Info    : Reading 'test-twist-3D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Circle)
Info    : Done meshing 1D (Wall 0.000701581s, CPU 0.0007s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0347133s, CPU 0.033609s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 351 nodes...
Info    : Done tetrahedrizing 359 nodes (Wall 0.00587228s, CPU 0.005386s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0115438s, CPU 0.008665s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.0

In [4]:
mat = FEM.material("body", E=2e5, ν=0)

Material("body", :Hooke, 200000.0, 0.0, 0.0, 100000.0, 66666.66666666667, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)

In [5]:
problem = FEM.Problem([mat])

Problem("test-twist-3D", :Solid, 3, 3, Material[Material("body", :Hooke, 200000.0, 0.0, 0.0, 100000.0, 66666.66666666667, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 2648, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [6]:
ux(x, y, z) = -z * y / 100000
uy(x, y, z) = z * x / 100000
uz(x, y, z) = z / 10000
f = FEM.field("body", fx=ux, fy=uy, fz=uz)
u = FEM.vectorField(problem, [f])

VectorField
[-2.4492935982947064e-18; -0.00999999999999999; … ; -0.0024129373526609733; 0.008770688268828654;;]

In [7]:
FEM.showDoFResults(u, :uvec)

1

In [8]:
n1x(x, y, z) = x
n1y(x, y, z) = y
n2x(x, y, z) = -y
n2y(x, y, z) = x
cs = FEM.CoordinateSystem([n1x, n1y, 0], [n2x, n2y, 0])
Q = FEM.rotateNodes(problem, "body", cs)

LowLevelFEM.Transformation(sparse([1, 2, 1, 2, 3, 4, 5, 4, 5, 6  …  7939, 7940, 7939, 7940, 7941, 7942, 7943, 7942, 7943, 7944], [1, 1, 2, 2, 3, 4, 4, 5, 5, 6  …  7939, 7939, 7940, 7940, 7941, 7942, 7942, 7943, 7943, 7944], [-1.0, 2.449293598294709e-16, -2.449293598294709e-16, -1.0, 1.0, -1.0, 2.4492935982947064e-16, -2.4492935982947064e-16, -1.0, 1.0  …  0.29415176552189454, 0.9557587241769506, -0.9557587241769506, 0.2941517655218945, 1.0, -0.9014841566908509, 0.4328121015352799, -0.4328121015352798, -0.9014841566908509, 1.0], 7944, 7944), 2648, 3)

In [9]:
FEM.showDoFResults(Q' * u, :uvec)

2

In [10]:
E = FEM.solveStrain(u)

TensorField
[[2.168404344971009e-19; 8.673617379884035e-19; … ; -2.2947987790176635e-6; 0.00010000000000000113;;], [2.710505431213761e-19; 2.710505431213761e-19; … ; -1.779658706575387e-5; 9.999999999999972e-5;;], [4.336808689942018e-19; 8.673617379884035e-19; … ; -1.2761178435102382e-5; 9.99999999999994e-5;;], [0.0; -1.0842021724855044e-19; … ; 7.975637002282537e-6; 9.99999999999995e-5;;], [-2.710505431213761e-19; -1.0842021724855044e-19; … ; 1.9105829146866618e-5; 9.999999999999994e-5;;], [-3.2526065174565133e-19; -2.168404344971009e-19; … ; 1.1481441538822032e-5; 9.999999999999983e-5;;], [1.0842021724855044e-19; 2.168404344971009e-19; … ; -2.526285831533294e-5; 9.99999999999994e-5;;], [-2.168404344971009e-19; 0.0; … ; -2.7692411135723724e-6; 9.999999999999983e-5;;], [-1.4181267808148018e-6; -8.116477483107484e-7; … ; 1.9105829146866672e-5; 0.00010000000000000018;;], [-3.3881317890172014e-21; 0.0; … ; 1.1481441538823117e-5; 0.00010000000000000026;;]  …  [-2.2910715222132505e-6; 5.325

In [11]:
E = FEM.elementsToNodes(E)

TensorField
[3.28791220023379e-8; -1.6199447148274298e-7; … ; -1.3755689854105446e-5; 9.999999999999885e-5;;]

In [12]:
FEM.showDoFResults(E, :ex)
FEM.showDoFResults(E, :ey)
FEM.showDoFResults(E, :ez)
FEM.showDoFResults(E, :exy)
FEM.showDoFResults(E, :eyz)
FEM.showDoFResults(E, :ezx)

8

In [13]:
FEM.showDoFResults(Q' * E * Q, :ex)
FEM.showDoFResults(Q' * E * Q, :ey)
FEM.showDoFResults(Q' * E * Q, :ez)
FEM.showDoFResults(Q' * E * Q, :exy)
FEM.showDoFResults(Q' * E * Q, :eyz)
FEM.showDoFResults(Q' * E * Q, :ezx)

14

In [14]:
S = FEM.solveStress(u)

TensorField
[[4.336808689942018e-14; 1.734723475976807e-13; … ; -0.4589597558035327; 20.000000000000227;;], [5.421010862427522e-14; 5.421010862427522e-14; … ; -3.559317413150774; 19.999999999999943;;], [8.673617379884035e-14; 1.734723475976807e-13; … ; -2.5522356870204765; 19.99999999999988;;], [0.0; -2.168404344971009e-14; … ; 1.5951274004565075; 19.9999999999999;;], [-5.421010862427522e-14; -2.168404344971009e-14; … ; 3.8211658293733235; 19.999999999999986;;], [-6.505213034913027e-14; -4.336808689942018e-14; … ; 2.2962883077644065; 19.999999999999964;;], [2.168404344971009e-14; 4.336808689942018e-14; … ; -5.052571663066589; 19.99999999999988;;], [-4.336808689942018e-14; 0.0; … ; -0.5538482227144745; 19.999999999999964;;], [-0.28362535616296036; -0.16232954966214969; … ; 3.8211658293733346; 20.000000000000036;;], [-6.776263578034403e-16; 0.0; … ; 2.2962883077646232; 20.000000000000053;;]  …  [-0.4582143044426501; 0.10650518326160707; … ; -2.2158353221506286; 20.000000000000053;;], [0.

In [15]:
S = FEM.elementsToNodes(S)

TensorField
[0.006575824400467581; -0.032398894296548596; … ; -2.7511379708210892; 19.99999999999977;;]

In [16]:
FEM.showDoFResults(S, :sx)
FEM.showDoFResults(S, :sy)
FEM.showDoFResults(S, :sz)
FEM.showDoFResults(S, :sxy)
FEM.showDoFResults(S, :syz)
FEM.showDoFResults(S, :szx)

20

In [17]:
FEM.showDoFResults(Q' * S * Q, :sx)
FEM.showDoFResults(Q' * S * Q, :sy)
FEM.showDoFResults(Q' * S * Q, :sz)
FEM.showDoFResults(Q' * S * Q, :sxy)
FEM.showDoFResults(Q' * S * Q, :syz)
FEM.showDoFResults(Q' * S * Q, :szx)

26

In [18]:
I = FEM.unitTensor(FEM.solveStrain(u))
I = FEM.elementsToNodes(I)

TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [19]:
FEM.probe(Q' * I * Q, 1, 1, 1)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [20]:
FEM.probe(FEM.solveStrain(u), 7, 7, 90)

3×3 Matrix{Float64}:
 -5.452e-7    -1.23e-8     -3.49377e-5
 -1.23e-8      6.202e-7     3.49355e-5
 -3.49377e-5   3.49355e-5   0.0001

In [21]:
FEM.probe(FEM.solveStress(u), 5, 5, 90)

3×3 Matrix{Float64}:
  0.165495   -0.0729841  -4.9406
 -0.0729841  -0.0272115   4.93822
 -4.9406      4.93822    20.0

In [22]:
FEM.probe(E, 7, 7, 90)

3×3 Matrix{Float64}:
 -3.248e-7    1.115e-7    -3.47485e-5
  1.115e-7    2.115e-7     3.47836e-5
 -3.47485e-5  3.47836e-5   0.0001

In [23]:
FEM.probe(S, 7, 7, 90)

3×3 Matrix{Float64}:
 -0.0649552  0.0223029  -6.94971
  0.0223029  0.0422954   6.95672
 -6.94971    6.95672    20.0

In [24]:
FEM.probe(FEM.solveStress(u), 7, 7, 90)

3×3 Matrix{Float64}:
 -0.109034    -0.00245901  -6.98754
 -0.00245901   0.124038     6.9871
 -6.98754      6.9871      20.0

In [25]:
FEM.probe(u, 7, 7, 90)

3-element Vector{Float64}:
 -0.0062998906
  0.0062998905
  0.009

In [26]:
FEM.probe(S, 7, 7, 90)

3×3 Matrix{Float64}:
 -0.0649552  0.0223029  -6.94971
  0.0223029  0.0422954   6.95672
 -6.94971    6.95672    20.0

In [27]:
F = 200000 * FEM.solveStrain(u) - FEM.solveStress(u)
FEM.showElementResults(F, :ex)
FEM.showElementResults(F, :ey)
FEM.showElementResults(F, :ez)
FEM.showElementResults(F, :exy)
FEM.showElementResults(F, :eyz)
FEM.showElementResults(F, :ezx)

32

In [28]:
FEM.probe(F, 10, 0, 100)

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [29]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [30]:
gmsh.finalize()